### Fine Tuning Whisper Small Model with afriSpeech-2000

reference:https://huggingface.co/blog/fine-tune-whisper?source=post_page-----976164a5eac8--------------------------------

afriSpeech-2000: https://huggingface.co/datasets/tobiolatunji/afrispeech-200

### maybe try english for tuning first

### different parameters

### African American = English + South African English

### Testing Whisper small without fine tuning - South African English Accent

In [1]:
from datasets import load_dataset, DatasetDict


In [2]:
afrispeech3 = DatasetDict()
afrispeech3['train'] = load_dataset("tobiolatunji/afrispeech-200", "south-african-english", split="train")
afrispeech3["test"] = load_dataset("tobiolatunji/afrispeech-200", "south-african-english", split="test")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 114it [00:00, 153204.31it/s]


Generating test split: 0 examples [00:00, ? examples/s]


Reading metadata...: 5it [00:00, 38409.38it/s]


Dataset afrispeech-200 downloaded and prepared to /Users/will/.cache/huggingface/datasets/tobiolatunji___afrispeech-200/south-african-english/1.0.0/93a9c8adc14618a3cbca8ef60b523ca2f8f18bb3998f3492c883f508faf1e0f2. Subsequent calls will reuse this data.


Found cached dataset afrispeech-200 (/Users/will/.cache/huggingface/datasets/tobiolatunji___afrispeech-200/south-african-english/1.0.0/93a9c8adc14618a3cbca8ef60b523ca2f8f18bb3998f3492c883f508faf1e0f2)


In [3]:
print(afrispeech3)

DatasetDict({
    train: Dataset({
        features: ['speaker_id', 'path', 'audio_id', 'audio', 'transcript', 'age_group', 'gender', 'accent', 'domain', 'country', 'duration'],
        num_rows: 114
    })
    test: Dataset({
        features: ['speaker_id', 'path', 'audio_id', 'audio', 'transcript', 'age_group', 'gender', 'accent', 'domain', 'country', 'duration'],
        num_rows: 5
    })
})


In [5]:
afrispeech3 = afrispeech3.remove_columns(['speaker_id', 'path', 'audio_id', 'age_group', 'gender', 'accent', 'domain', 'country', 'duration'])

In [11]:
print(afrispeech3)

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcript'],
        num_rows: 114
    })
    test: Dataset({
        features: ['audio', 'transcript'],
        num_rows: 5
    })
})


In [15]:
import torch
device = torch.device("mps")

In [23]:
from transformers import WhisperFeatureExtractor, WhisperProcessor
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor
from jiwer import wer
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

processor = WhisperProcessor.from_pretrained("openai/whisper-small")

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)


In [32]:
# def map_to_pred(batch):
#     audio = batch["audio"]
#     input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
#     batch["reference"] = processor.tokenizer._normalize(batch['transcript'])

#     with torch.no_grad():
#         predicted_ids = model.generate(input_features.to(device))[0]
#     transcription = processor.decode(predicted_ids)
#     batch["prediction"] = processor.tokenizer._normalize(transcription)
#     return batch
import librosa


def map_to_pred(batch):
    # Resample audio from 44100 Hz to 16000 Hz
    audio_resampled = librosa.resample(batch["audio"]["array"], orig_sr=batch["audio"]["sampling_rate"], target_sr=16000)
    
    # Process resampled audio
    input_features = processor(audio_resampled, sampling_rate=16000, return_tensors="pt").input_features
    batch["reference"] = processor.tokenizer._normalize(batch['transcript'])

    with torch.no_grad():
        predicted_ids = model.generate(input_features.to(device))[0]
    transcription = processor.decode(predicted_ids)
    batch["prediction"] = processor.tokenizer._normalize(transcription)
    return batch


In [30]:
afrispeech3 = afrispeech3.map(lambda batch: {'audio': batch['audio'], 'transcript': batch['transcript']})
# Apply the mapping function
afrispeech3["test"] = afrispeech3["test"].map(map_to_pred)
afrispeech3["train"] = afrispeech3["train"].map(map_to_pred)

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [34]:
from evaluate import load
wer = load("wer")
test_result = 100 * wer.compute(references=afrispeech3["test"]["reference"], predictions=afrispeech3["test"]["prediction"])
train_result = 100 * wer.compute(references=afrispeech3["train"]["reference"], predictions=afrispeech3["train"]["prediction"])

print(f"WER on test set: {test_result}%")
print(f"WER on train set: {train_result}%")

WER on test set: 25.53191489361702%
WER on train set: 21.236059479553905%


### Testing Whisper small without fine tuning - English Accent

In [83]:
afrispeech2 = DatasetDict()
afrispeech2['train'] = load_dataset("tobiolatunji/afrispeech-200", "english", split="validation")

Found cached dataset afrispeech-200 (/Users/will/.cache/huggingface/datasets/tobiolatunji___afrispeech-200/english/1.0.0/93a9c8adc14618a3cbca8ef60b523ca2f8f18bb3998f3492c883f508faf1e0f2)


In [84]:
afrispeech2["test"] = load_dataset("tobiolatunji/afrispeech-200", "english", split="test")

Found cached dataset afrispeech-200 (/Users/will/.cache/huggingface/datasets/tobiolatunji___afrispeech-200/english/1.0.0/93a9c8adc14618a3cbca8ef60b523ca2f8f18bb3998f3492c883f508faf1e0f2)


In [85]:
afrispeech2 = afrispeech2.remove_columns(['speaker_id', 'path', 'audio_id', 'age_group', 'gender', 'accent', 'domain', 'country', 'duration'])

In [66]:
print(afrispeech2)

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcript'],
        num_rows: 44
    })
    test: Dataset({
        features: ['audio', 'transcript'],
        num_rows: 106
    })
})


In [40]:
afrispeech2 = afrispeech2.map(lambda batch: {'audio': batch['audio'], 'transcript': batch['transcript']})
# Apply the mapping function
afrispeech2["test"] = afrispeech2["test"].map(map_to_pred)
afrispeech2["train"] = afrispeech2["train"].map(map_to_pred)

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

Map:   0%|          | 0/106 [00:00<?, ? examples/s]

Map:   0%|          | 0/106 [00:00<?, ? examples/s]

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

In [41]:
test_result2 = 100 * wer.compute(references=afrispeech2["test"]["reference"], predictions=afrispeech2["test"]["prediction"])
train_result2 = 100 * wer.compute(references=afrispeech2["train"]["reference"], predictions=afrispeech2["train"]["prediction"])

print(f"WER on test set: {test_result2}%")
print(f"WER on train set: {train_result2}%")

WER on test set: 49.13083257090577%
WER on train set: 31.541725601131542%


### Summary
Without fine-tuning, using Whisper-small on South African English Accent data gives us WER of 25% on testing data and 21% on training data. On English Accent data, we got a WER of 49% on test data and 31% on train (fewer samples), which is surprising.

### Fine Tuning with English Accent

In [42]:
from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="en", task="transcribe")

In [86]:
input_str = afrispeech2["test"][0]["transcript"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 Noted to have intermittent snorting sounds with respirations but no upper airway secretions this past shift.
Decoded w/ special:    <|startoftranscript|><|en|><|transcribe|><|notimestamps|>Noted to have intermittent snorting sounds with respirations but no upper airway secretions this past shift.<|endoftext|>
Decoded w/out special: Noted to have intermittent snorting sounds with respirations but no upper airway secretions this past shift.
Are equal:             True


In [87]:
print(afrispeech2["test"][0])

{'audio': {'path': '/Users/will/.cache/huggingface/datasets/downloads/extracted/bfccd4aad94d30ba1280e3e1833e7f21cc6cc6a2c585d219d72cd43be0a4d8e3/763f3b6b-f7f9-43ac-8ea0-55d48c802dde/cbd6e3693a746386fe41eb649d144249.wav', 'array': array([ 0.0012207 ,  0.00119019,  0.00112915, ..., -0.00167847,
       -0.00067139,  0.00082397]), 'sampling_rate': 44100}, 'transcript': 'Noted to have intermittent snorting sounds with respirations but no upper airway secretions this past shift.'}


In [88]:
from datasets import Audio, load_dataset
afrispeech2 = afrispeech2.cast_column("audio", Audio(sampling_rate=16000))
print(afrispeech2["test"][0])

{'audio': {'path': '/Users/will/.cache/huggingface/datasets/downloads/extracted/bfccd4aad94d30ba1280e3e1833e7f21cc6cc6a2c585d219d72cd43be0a4d8e3/763f3b6b-f7f9-43ac-8ea0-55d48c802dde/cbd6e3693a746386fe41eb649d144249.wav', 'array': array([ 0.00079288,  0.00117098,  0.00097301, ...,  0.00132558,
       -0.00011966, -0.00118353]), 'sampling_rate': 16000}, 'transcript': 'Noted to have intermittent snorting sounds with respirations but no upper airway secretions this past shift.'}


In [89]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["transcript"], padding="max_length", truncation=True, max_length=448).input_ids
    return batch

In [90]:
afrispeech2 = afrispeech2.map(prepare_dataset, remove_columns=afrispeech2.column_names["test"], num_proc=4)


Map (num_proc=4):   0%|          | 0/44 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/106 [00:00<?, ? examples/s]

In [91]:
print(afrispeech2)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 44
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 106
    })
})


In [92]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [93]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [102]:
import evaluate
metric = evaluate.load("wer")

In [103]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [104]:
from transformers import WhisperForConditionalGeneration

model_f = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model_f.config.forced_decoder_ids = None
model_f.config.suppress_tokens = []


In [105]:
model_f.to('mps')

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [106]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/Users/will/Documents/GitHub/Bias-in-ASR/model-weights",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=100, # original 500
    max_steps=100, # original 2000
    gradient_checkpointing=True,
    fp16=False,# no gpu here. only for NVDIA if set true
    evaluation_strategy="steps",
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=50, # original 1000
    eval_steps=50, # original 1000
    logging_steps=10, # original 10
    load_best_model_at_end=True,#early stopping
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False, 
    #if one does not want to upload the model checkpoints to the Hub, set push_to_hub=False.
)

In [107]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=afrispeech2["test"],
    eval_dataset=afrispeech2["train"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [108]:
trainer.train()

Step,Training Loss,Validation Loss,Wer
50,0.643900,0.302495,137.714286
100,0.426500,0.267302,65.571429


/Users/will/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=100, training_loss=0.516513934135437, metrics={'train_runtime': 11985.0039, 'train_samples_per_second': 0.134, 'train_steps_per_second': 0.008, 'total_flos': 4.3749546688512e+17, 'train_loss': 0.516513934135437, 'epoch': 14.29})

In [1]:
from datasets import load_dataset, DatasetDict

afrispeech = DatasetDict()

afrispeech['train'] = load_dataset("tobiolatunji/afrispeech-200", "isizulu", split="train")

afrispeech["test"] = load_dataset("tobiolatunji/afrispeech-200", "isizulu", split="test")

print(afrispeech)

Found cached dataset afrispeech-200 (/Users/will/.cache/huggingface/datasets/tobiolatunji___afrispeech-200/isizulu/1.0.0/93a9c8adc14618a3cbca8ef60b523ca2f8f18bb3998f3492c883f508faf1e0f2)
Found cached dataset afrispeech-200 (/Users/will/.cache/huggingface/datasets/tobiolatunji___afrispeech-200/isizulu/1.0.0/93a9c8adc14618a3cbca8ef60b523ca2f8f18bb3998f3492c883f508faf1e0f2)


DatasetDict({
    train: Dataset({
        features: ['speaker_id', 'path', 'audio_id', 'audio', 'transcript', 'age_group', 'gender', 'accent', 'domain', 'country', 'duration'],
        num_rows: 775
    })
    test: Dataset({
        features: ['speaker_id', 'path', 'audio_id', 'audio', 'transcript', 'age_group', 'gender', 'accent', 'domain', 'country', 'duration'],
        num_rows: 100
    })
})


In [2]:
afrispeech = afrispeech.remove_columns(['speaker_id', 'path', 'audio_id', 'age_group', 'gender', 'accent', 'domain', 'country', 'duration'])

In [3]:
afrispeech

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcript'],
        num_rows: 775
    })
    test: Dataset({
        features: ['audio', 'transcript'],
        num_rows: 100
    })
})

Load WhisperFeatureExtractor

Speech is represented by a 1-dimensional array that varies with time. The value of the array at any given time step is the signal's amplitude at that point. From the amplitude information alone, we can reconstruct the frequency spectrum of the audio and recover all acoustic features.

passing audio with the wrong sampling rate can falter an ASR model that expects one sampling rate and receives another. The Whisper feature extractor expects audio inputs with a sampling rate of 16kHz, so we need to match our inputs to this value.

The Whisper feature extractor performs two operations. It first pads/truncates a batch of audio samples such that all samples have an input length of 30s. The second operation that the Whisper feature extractor performs is converting the padded audio arrays to log-Mel spectrograms. These spectrograms are a visual representation of the frequencies of a signal, rather like a Fourier transform. a

In [4]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")


In [5]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="en", task="transcribe")


In [6]:
input_str = afrispeech["train"][0]["transcript"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 The hedge, which he started building on March 3, cost him roughly 27 million and scored big as stock and debt markets floundered on fears of the coming pandemic  fears, critics say, that he helped stoke.

Decoded w/ special:    <|startoftranscript|><|en|><|transcribe|><|notimestamps|>The hedge, which he started building on March 3, cost him roughly 27 million and scored big as stock and debt markets floundered on fears of the coming pandemic  fears, critics say, that he helped stoke.
<|endoftext|>
Decoded w/out special: The hedge, which he started building on March 3, cost him roughly 27 million and scored big as stock and debt markets floundered on fears of the coming pandemic  fears, critics say, that he helped stoke.

Are equal:             True


In [7]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="en", task="transcribe")


In [8]:
print(afrispeech["train"][0])

{'audio': {'path': '/Users/will/.cache/huggingface/datasets/downloads/extracted/50dcc66e28367290d02ab2ea56d0d787fb3efd769ecf196eabf5b403403a5420/f36f9ceb-5b90-4abe-966d-75f8aa70b300/bd154e9a0506c2636a80b98b25ced4fb.wav', 'array': array([ 3.05175781e-03,  2.74658203e-03,  2.68554688e-03, ...,
       -2.13623047e-04, -3.05175781e-05,  1.83105469e-04]), 'sampling_rate': 44100}, 'transcript': 'The hedge, which he started building on March 3, cost him roughly 27 million and scored big as stock and debt markets floundered on fears of the coming pandemic  fears, critics say, that he helped stoke.\n'}


Since our input audio is sampled at 44.1kHz, we need to downsample it to 16kHz before passing it to the Whisper feature extractor.

This operation does not change the audio in-place, but rather signals to datasets to resample audio samples on the fly the first time that they are loaded

In [9]:
import librosa

In [10]:
print(librosa.__version__)

0.10.1


In [11]:
from datasets import Audio, load_dataset

afrispeech = afrispeech.cast_column("audio", Audio(sampling_rate=16000))

In [12]:
print(afrispeech["train"][0])

{'audio': {'path': '/Users/will/.cache/huggingface/datasets/downloads/extracted/50dcc66e28367290d02ab2ea56d0d787fb3efd769ecf196eabf5b403403a5420/f36f9ceb-5b90-4abe-966d-75f8aa70b300/bd154e9a0506c2636a80b98b25ced4fb.wav', 'array': array([ 1.84984040e-03,  3.12240026e-03,  3.18585220e-03, ...,
       -1.30385975e-03, -7.35237671e-04, -8.48602285e-05]), 'sampling_rate': 16000}, 'transcript': 'The hedge, which he started building on March 3, cost him roughly 27 million and scored big as stock and debt markets floundered on fears of the coming pandemic  fears, critics say, that he helped stoke.\n'}


Training and Evaluation

Define a data collator: the data collator takes our pre-processed data and prepares PyTorch tensors ready for the model.

In [13]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["transcript"]).input_ids
    return batch


In [14]:
afrispeech = afrispeech.map(prepare_dataset, remove_columns=afrispeech.column_names["train"], num_proc=4)



Map (num_proc=4):   0%|          | 0/775 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

In [15]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [16]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

Evaluation Metrics

In [17]:
import evaluate

metric = evaluate.load("wer")

/Users/will/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/will/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


We then simply have to define a function that takes our model predictions and returns the WER metric. This function, called compute_metrics, first replaces -100 with the pad_token_id in the label_ids (undoing the step we applied in the data collator to ignore padded tokens correctly in the loss). It then decodes the predicted and label ids to strings. Finally, it computes the WER between the predictions and reference labels:

In [18]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


Load a Pre-Trained Checkpoint


In [19]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []


Define the Training Arguments


In [20]:
#!pip install accelerate -U

In [21]:
import torch

device = "mps" if torch.backends.mps.is_available() else "cpu"  # Use MPS if available, else fall back to CPU
print(f"Using device: {device}")


Using device: mps


In [22]:
model.to(device)


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [23]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/Users/will/Documents/GitHub/Bias-in-ASR/model-weights",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=100, # original 500
    max_steps=400, # original 2000
    gradient_checkpointing=True,
    fp16=False,# no gpu here. only for NVDIA if set true
    evaluation_strategy="steps",
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=50, # original 1000
    eval_steps=50, # original 1000
    logging_steps=10, # original 10
    load_best_model_at_end=True,#early stopping
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False, 
    #if one does not want to upload the model checkpoints to the Hub, set push_to_hub=False.
)
# training_args = Seq2SeqTrainingArguments(
#     output_dir="/Users/will/Documents/GitHub/Bias-in-ASR/model-weights",  # change to a repo name of your choice
#     per_device_train_batch_size=16,
#     gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
#     learning_rate=1e-5,
#     warmup_steps=500,
#     max_steps=4000,
#     gradient_checkpointing=True,
#     fp16=False,
#     evaluation_strategy="steps",
#     per_device_eval_batch_size=8,
#     predict_with_generate=True,
#     generation_max_length=225,
#     save_steps=1000,
#     eval_steps=1000,
#     logging_steps=25,
#     load_best_model_at_end=True,
#     metric_for_best_model="wer",
#     greater_is_better=False,
#     push_to_hub=False,
# )

In [31]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hi",  # Your chosen directory
    per_device_train_batch_size=8,  # Adjusted to desired train batch size
    per_device_eval_batch_size=8,  # Adjusted to desired eval batch size
    gradient_accumulation_steps=2,  # Adjust as needed, considering your hardware and batch size
    learning_rate=1e-5,  # As specified
    warmup_steps=500,  # As specified
    max_steps=1000,  # Changed to training_steps
    fp16=False,  # Enable mixed precision training (assuming you're using a compatible GPU)
    evaluation_strategy="steps",
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,  # You might want to adjust this based on your total training steps to ensure you're saving periodically
    eval_steps=1000,  # Adjust according to your preferences and dataset size
    logging_steps=25,  # As before
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,  # Set based on whether you want to push to the hub
    # Additional parameters for optimizer and scheduler
    optim="adamw_hf",  # Using AdamW optimizer from HF, which matches your request for Adam
    lr_scheduler_type="linear",  # As specified
    # New arguments for seed and report_to are not directly part of Seq2SeqTrainingArguments; see notes below.
)

In [32]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model_f,
    train_dataset=afrispeech["train"],
    eval_dataset=afrispeech["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    
)


/Users/will/anaconda3/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [25]:
trainer.train()

/Users/will/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
50,0.000000,nan,100.000000
100,0.000000,nan,100.000000
150,0.000000,nan,100.000000
200,0.000000,nan,100.000000
250,0.000000,nan,100.000000
300,0.000000,nan,100.000000
350,0.000000,nan,100.000000
400,0.000000,nan,100.000000


/Users/will/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/Users/will/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/Users/will/anaconda3/lib/python3.11/site-packages/torch/utils/check

TrainOutput(global_step=400, training_loss=0.23362467288970948, metrics={'train_runtime': 7696.5511, 'train_samples_per_second': 0.832, 'train_steps_per_second': 0.052, 'total_flos': 1.84463787884544e+18, 'train_loss': 0.23362467288970948, 'epoch': 8.25})

In [26]:
#!mkdir -p /Users/will/.cache/huggingface/metrics/wer/default


In [28]:
print(len(afrispeech["train"]))
print(len(afrispeech["test"]))

775
100


In [ ]:
#second try with another set of parameters

In [33]:
trainer.train()

/Users/will/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/Users/will/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


RuntimeError: MPS backend out of memory (MPS allocated: 11.79 GB, other allocations: 8.02 GB, max allowed: 20.40 GB). Tried to allocate 823.97 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).